## Assignment # 2

For this assignment, I've choosen the following images:

    1. 1.jpg
    2. 2.jpg
    3. 3.jpg
    4. 4.jpg
    5. 7.jpg

I've divided the my Jupyter NoteBook into following sections

    1. Helper code
    2. Image reading and resizing
    3. Image patching, and histogram calculation
    4. K Means algorithm implementation
    5. Mean Shift algorithm
    6. Super Pixel Segmentation
    7. Active Contouring (using Canny edge detectation)
    8. Otsu thresholding

In [17]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, MeanShift
from skimage.segmentation import slic, mark_boundaries
import random


class ImagePatcher(object):
    def __init__(self, image, dimensions):
        self.image = image
        self.dimensions = dimensions
        self.N, self.M = dimensions  # N = # of rows, M = # of columns
        self.num_rows, self.num_cols = self.get_patch_rows_and_columns()
        self.image_patch_list = []
        self.hist_list = np.empty((self.N * self.M, 256))

    def get_patch_rows_and_columns(self):
        height, width, _ = self.image.shape

        # Calculate the number of rows and columns for a patch
        num_rows = height // self.N
        num_cols = width // self.M

        return (num_rows, num_cols)

    def split_image(self, dir_name, save=False):
        # Iterate through the large image and divide it into smaller sub-images
        for row in range(self.N):
            temp = []
            for col in range(self.M):
                # Calculate the coordinates for each sub-image
                x_start = col * self.num_cols
                x_end = (col + 1) * self.num_cols
                y_start = row * self.num_rows
                y_end = (row + 1) * self.num_rows

                sub_image = self.image[y_start:y_end, x_start:x_end].astype(np.float32)
                temp.append(sub_image)

                if save:
                    cv.imwrite(
                        f"./results/image-patches/{dir_name}/image-patch-{row + 1}-{col + 1}.jpg",
                        sub_image,
                    )

            self.image_patch_list.append(temp)

        return self.image_patch_list

    def merge_image_patches(self):
        canvas = np.zeros_like(self.image, dtype=np.uint8)

        for row in range(self.N):
            for col in range(self.M):
                x_start = col * self.num_cols
                x_end = (col + 1) * self.num_cols
                y_start = row * self.num_rows
                y_end = (row + 1) * self.num_rows
                patch = self.image_patch_list[row][col]
                print("patch", patch)

                canvas[y_start:y_end, x_start:x_end] = patch

        return canvas

    def get_histograms(self):
        count = 0
        for row in range(self.N):
            for col in range(self.M):
                patch = self.image_patch_list[row][col]
                hist = cv.calcHist([patch], [0], None, [256], [0, 256])
                self.hist_list[count, :] = hist.reshape(256)
                count += 1

        return self.hist_list

    def gen_color_list(self, k):
        # random.seed(k)

        color_list = [
            (
                random.randrange(10, 255),
                random.randrange(10, 255),
                random.randrange(10, 255),
            )
            for _ in range(k)
        ]
        return color_list

    def colorize_and_merge_patch(self, cluster_res):
        colored_image = np.zeros_like(self.image)
        cluster_colors = self.gen_color_list(len(cluster_res.labels_))
        labels = cluster_res.labels_

        for row in range(self.N):
            for col in range(self.M):
                x_start = col * self.num_cols
                x_end = (col + 1) * self.num_cols
                y_start = row * self.num_rows
                y_end = (row + 1) * self.num_rows

                color = cluster_colors[labels[row + col]]
                patch = self.image_patch_list[row][col]
                patch = cv.cvtColor(patch, cv.COLOR_BGR2RGB)
                patch[:] = color 

                colored_image[y_start:y_end, x_start:x_end] = patch

        return colored_image


def get_image_contour(image, a=125, b=125):
    canny = cv.Canny(image, a, b)
    contours_list, _ = cv.findContours(canny, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
    blank_img = np.zeros(image.shape, dtype="uint8")
    cv.drawContours(blank_img, contours_list, -1, (255, 255, 255), 1)

    return blank_img

In [8]:
# image reading and resizing
image_1 = cv.imread("./Dataset/1.jpg")
image_2 = cv.imread("./Dataset/2.jpg")
image_3 = cv.imread("./Dataset/3.jpg")
image_4 = cv.imread("./Dataset/4.jpg")
image_7 = cv.imread("./Dataset/7.jpg")

image_1 = cv.resize(image_1, (1300, 700))
image_2 = cv.resize(image_2, (1300, 700))
image_3 = cv.resize(image_3, (1300, 700))
image_4 = cv.resize(image_4, (1300, 700))
image_7 = cv.resize(image_7, (1300, 700))

In [4]:
image_patch_1 = ImagePatcher(image_1, (5, 5))
image_patch_2 = ImagePatcher(image_2, (6, 6))
image_patch_3 = ImagePatcher(image_3, (10, 10))
image_patch_4 = ImagePatcher(image_4, (12, 11))
image_patch_7 = ImagePatcher(image_7, (20, 15))

# generate patch for all images
image_patch_list_1 = image_patch_1.split_image("image-1",True)
image_patch_list_2 = image_patch_2.split_image("image-2",True)
image_patch_list_3 = image_patch_3.split_image("image-3",True)
image_patch_list_4 = image_patch_4.split_image("image-4",True)
image_patch_list_7 = image_patch_7.split_image("image-7",True)

# patch level histograms
image_patch_histograms_1 = image_patch_1.get_histograms()
image_patch_histograms_2 = image_patch_2.get_histograms()
image_patch_histograms_3 = image_patch_3.get_histograms()
image_patch_histograms_4 = image_patch_4.get_histograms()
image_patch_histograms_7 = image_patch_7.get_histograms()

In [20]:
# k-means algo
k = 4
criteria = (cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv.KMEANS_RANDOM_CENTERS

image_patch_histograms_kmeans_1 = KMeans(
    n_clusters=k, init="random", random_state=0, n_init="auto"
).fit(image_patch_histograms_1)

image_patch_histograms_kmeans_2 = KMeans(
    n_clusters=k + 2, init="random", random_state=0, n_init="auto"
).fit(image_patch_histograms_2)

image_patch_histograms_kmeans_3 = KMeans(
    n_clusters=k + 4, init="random", random_state=0, n_init="auto"
).fit(image_patch_histograms_3)

image_patch_histograms_kmeans_4 = KMeans(
    n_clusters=k + 6, init="random", random_state=0, n_init="auto"
).fit(image_patch_histograms_4)

image_patch_histograms_kmeans_7 = KMeans(
    n_clusters=k + 8, init="random", random_state=0, n_init="auto"
).fit(image_patch_histograms_7)

colored_output_1 = image_patch_1.colorize_and_merge_patch(
    image_patch_histograms_kmeans_1
)
colored_output_2 = image_patch_2.colorize_and_merge_patch(
    image_patch_histograms_kmeans_2
)
colored_output_3 = image_patch_3.colorize_and_merge_patch(
    image_patch_histograms_kmeans_3
)
colored_output_4 = image_patch_4.colorize_and_merge_patch(
    image_patch_histograms_kmeans_4
)
colored_output_7 = image_patch_7.colorize_and_merge_patch(
    image_patch_histograms_kmeans_7
)

cv.imwrite("results/kmeans-clustering/kmeans-image-colored-res-1.jpg", colored_output_1)
cv.imwrite("results/kmeans-clustering/kmeans-image-colored-res-2.jpg", colored_output_2)
cv.imwrite("results/kmeans-clustering/kmeans-image-colored-res-3.jpg", colored_output_3)
cv.imwrite("results/kmeans-clustering/kmeans-image-colored-res-4.jpg", colored_output_4)
cv.imwrite("results/kmeans-clustering/kmeans-image-colored-res-7.jpg", colored_output_7)

# cv.imshow("colored output 1", colored_output_1)
# cv.imshow("colored output 2", colored_output_2)
# cv.imshow("colored output 3", colored_output_3)
# cv.imshow("colored output 4", colored_output_4)
# cv.imshow("colored output 7", colored_output_7)


cv.waitKey(0)
cv.destroyAllWindows()

In [22]:
# mean shift algoritm
bw = 2
image_patch_meanshift_1 = MeanShift(bandwidth=bw).fit(image_patch_histograms_1)
image_patch_meanshift_2 = MeanShift(bandwidth=bw).fit(image_patch_histograms_2)
image_patch_meanshift_3 = MeanShift(bandwidth=bw).fit(image_patch_histograms_3)
image_patch_meanshift_4 = MeanShift(bandwidth=bw).fit(image_patch_histograms_4)
image_patch_meanshift_7 = MeanShift(bandwidth=bw).fit(image_patch_histograms_7)

colored_output_ms_1 = image_patch_1.colorize_and_merge_patch(image_patch_meanshift_1)
colored_output_ms_2 = image_patch_2.colorize_and_merge_patch(image_patch_meanshift_2)
colored_output_ms_3 = image_patch_3.colorize_and_merge_patch(image_patch_meanshift_3)
colored_output_ms_4 = image_patch_4.colorize_and_merge_patch(image_patch_meanshift_4)
colored_output_ms_7 = image_patch_7.colorize_and_merge_patch(image_patch_meanshift_7)

cv.imwrite("results/mean-shift/mean-shift-colored-res-1.jpg", colored_output_ms_1)
cv.imwrite("results/mean-shift/mean-shift-colored-res-2.jpg", colored_output_ms_2)
cv.imwrite("results/mean-shift/mean-shift-colored-res-3.jpg", colored_output_ms_3)
cv.imwrite("results/mean-shift/mean-shift-colored-res-5.jpg", colored_output_ms_4)
cv.imwrite("results/mean-shift/mean-shift-colored-res-7.jpg", colored_output_ms_7)

True

In [ ]:
# active contouring
contour_image_1 = get_image_contour(image_1)
contour_image_2 = get_image_contour(image_2)
contour_image_3 = get_image_contour(image_3)
contour_image_4 = get_image_contour(image_4)
contour_image_7 = get_image_contour(image_7)

# cv.imshow("image 1 active contour", contour_image_1)
# cv.imshow("image 2 active contour", contour_image_2)
# cv.imshow("image 3 active contour", contour_image_3)
# cv.imshow("image 4 active contour", contour_image_4)
# cv.imshow("image 7 active contour", contour_image_7)

cv.imwrite("results/active-contouring/active-contour-1.jpg", contour_image_1)
cv.imwrite("results/active-contouring/active-contour-2.jpg", contour_image_2)
cv.imwrite("results/active-contouring/active-contour-3.jpg", contour_image_3)
cv.imwrite("results/active-contouring/active-contour-4.jpg", contour_image_4)
cv.imwrite("results/active-contouring/active-contour-7.jpg", contour_image_7)

cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# otsu thresholding
_, image_thresh_ostu_1 = cv.threshold(
    cv.cvtColor(image_1, cv.COLOR_BGR2GRAY), 120, 255, cv.THRESH_BINARY + cv.THRESH_OTSU
)
_, image_thresh_ostu_2 = cv.threshold(
    cv.cvtColor(image_2, cv.COLOR_BGR2GRAY), 120, 255, cv.THRESH_BINARY + cv.THRESH_OTSU
)
_, image_thresh_ostu_3 = cv.threshold(
    cv.cvtColor(image_3, cv.COLOR_BGR2GRAY), 120, 255, cv.THRESH_BINARY + cv.THRESH_OTSU
)
_, image_thresh_ostu_4 = cv.threshold(
    cv.cvtColor(image_4, cv.COLOR_BGR2GRAY), 150, 255, cv.THRESH_BINARY + cv.THRESH_OTSU
)
_, image_thresh_ostu_7 = cv.threshold(
    cv.cvtColor(image_7, cv.COLOR_BGR2GRAY), 120, 255, cv.THRESH_BINARY + cv.THRESH_OTSU
)

# cv.imshow("image 1 otsu threshold", image_thresh_ostu_1)
# cv.imshow("image 2 otsu threshold", image_thresh_ostu_2)
# cv.imshow("image 3 otsu threshold", image_thresh_ostu_3)
# cv.imshow("image 4 otsu threshold", image_thresh_ostu_4)
# cv.imshow("image 7 otsu threshold", image_thresh_ostu_7)

cv.imwrite("results/otsu-thresholding/otstu-thresholding-1.jpg", image_thresh_ostu_1)
cv.imwrite("results/otsu-thresholding/otstu-thresholding-2.jpg", image_thresh_ostu_2)
cv.imwrite("results/otsu-thresholding/otstu-thresholding-3.jpg", image_thresh_ostu_3)
cv.imwrite("results/otsu-thresholding/otstu-thresholding-4.jpg", image_thresh_ostu_4)
cv.imwrite("results/otsu-thresholding/otstu-thresholding-7.jpg", image_thresh_ostu_7)

cv.waitKey(0)
cv.destroyAllWindows()